<br>

# Introdução

In [1]:
import os
import re
import time
import json
import folium
import random
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from folium import plugins
from osgeo import gdal, osr
from bs4 import BeautifulSoup
from tqdm.notebook import trange, tqdm

<br>

# Read and Join data

In [2]:
# Lê o arquivo csv com o nome dos municípios
df = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp_mpsp/main/data/tabs/output/tab_municipio_gaema_rp.csv',
)

# Lê o arquivo gejson com delimitação dos municípios
gdf = gpd.read_file(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/shps/sp_250k_wgs84.geojson',
)
gdf.drop(['municipio_nome'], axis=1, inplace=True)
gdf['id_municipio'] = gdf['id_municipio'].astype(int)
gdf['geometry'] = gdf.simplify(0.0015)

# Merge
gdf = gdf.merge(
    df,
    on='id_municipio',
    how='left'
)

# Results
gdf.head()

id_municipio                                           geometry  \
0       3500105  POLYGON ((-51.05425 -21.40465, -51.04624 -21.4...   
1       3500204  POLYGON ((-49.65795 -21.20333, -49.65361 -21.2...   
2       3500303  POLYGON ((-46.97640 -21.96818, -46.97399 -21.9...   
3       3500402  POLYGON ((-46.73501 -21.81891, -46.72104 -21.8...   
4       3500501  POLYGON ((-46.60614 -22.44173, -46.59592 -22.4...   

     municipio_nome  id_gaema  id_redeprotetiva num_gaema gaema_nome  \
0        Adamantina       NaN              11.0       NaN        NaN   
1            Adolfo       NaN              10.0       NaN        NaN   
2             Aguaí       NaN              14.0       NaN        NaN   
3    Águas da Prata       NaN              14.0       NaN        NaN   
4  Águas de Lindóia       NaN              14.0       NaN        NaN   

  gaema_secretario gaema_secretario_funcao gaema_endereco  ... gaema_cep  \
0              NaN                     NaN            NaN  ...       NaN   
1              NaN                     NaN            NaN  ...       NaN   
2              NaN                     NaN            NaN  ...       NaN   
3              NaN                     NaN            NaN  ...       NaN   
4              NaN                     NaN            NaN  ...       NaN   

  gaema_municipio gaema_telefone gaema_email gaema_link gaema_ato_criacao  \
0             NaN            NaN         NaN        NaN               NaN   
1             NaN            NaN         NaN        NaN               NaN   
2             NaN            NaN         NaN        NaN               NaN   
3             NaN            NaN         NaN        NaN               NaN   
4             NaN            NaN         NaN        NaN               NaN   

  gaema_ato_criacao_link rp_num        rp_nome             rp_ugrhi  
0                    NaN     XI  Aguapeí/Peixe  UGRHI Aguapeí/Peixe  
1                    NaN      X    Baixo Tietê    UGRHI Baixo Tietê  
2                    NaN    XIV      Alto Mogi     UGRHI Mogi-Guaçu  
3                    NaN    XIV      Alto Mogi     UGRHI Mogi-Guaçu  
4                    NaN    XIV      Alto Mogi     UGRHI Mogi-Guaçu  

[5 rows x 23 columns]

<br>

# Adjust and Save data

In [3]:
# Save geojson
gdf.to_file(
    os.path.join('data', 'shps', 'sp_mpsp.geojson'),
    driver='GeoJSON',
    encoding='utf-8'
)

# Results
gdf.head()

id_municipio                                           geometry  \
0       3500105  POLYGON ((-51.05425 -21.40465, -51.04624 -21.4...   
1       3500204  POLYGON ((-49.65795 -21.20333, -49.65361 -21.2...   
2       3500303  POLYGON ((-46.97640 -21.96818, -46.97399 -21.9...   
3       3500402  POLYGON ((-46.73501 -21.81891, -46.72104 -21.8...   
4       3500501  POLYGON ((-46.60614 -22.44173, -46.59592 -22.4...   

     municipio_nome  id_gaema  id_redeprotetiva num_gaema gaema_nome  \
0        Adamantina       NaN              11.0       NaN        NaN   
1            Adolfo       NaN              10.0       NaN        NaN   
2             Aguaí       NaN              14.0       NaN        NaN   
3    Águas da Prata       NaN              14.0       NaN        NaN   
4  Águas de Lindóia       NaN              14.0       NaN        NaN   

  gaema_secretario gaema_secretario_funcao gaema_endereco  ... gaema_cep  \
0              NaN                     NaN            NaN  ...       NaN   
1              NaN                     NaN            NaN  ...       NaN   
2              NaN                     NaN            NaN  ...       NaN   
3              NaN                     NaN            NaN  ...       NaN   
4              NaN                     NaN            NaN  ...       NaN   

  gaema_municipio gaema_telefone gaema_email gaema_link gaema_ato_criacao  \
0             NaN            NaN         NaN        NaN               NaN   
1             NaN            NaN         NaN        NaN               NaN   
2             NaN            NaN         NaN        NaN               NaN   
3             NaN            NaN         NaN        NaN               NaN   
4             NaN            NaN         NaN        NaN               NaN   

  gaema_ato_criacao_link rp_num        rp_nome             rp_ugrhi  
0                    NaN     XI  Aguapeí/Peixe  UGRHI Aguapeí/Peixe  
1                    NaN      X    Baixo Tietê    UGRHI Baixo Tietê  
2                    NaN    XIV      Alto Mogi     UGRHI Mogi-Guaçu  
3                    NaN    XIV      Alto Mogi     UGRHI Mogi-Guaçu  
4                    NaN    XIV      Alto Mogi     UGRHI Mogi-Guaçu  

[5 rows x 23 columns]

<br>

# Layers

In [4]:
def add_lyr_cartodbpositron(min_zoom, max_zoom):
    lyr = folium.TileLayer(        
        tiles='cartodbpositron',
        name='CartoDB Positron',
        min_zoom=min_zoom,
        max_zoom=max_zoom,        
        overlay=False,
        control=False,
        show=False,
    )
    return lyr

In [5]:
def add_lyr_gaema():
    # Input
    gdf = gpd.read_file(os.path.join('data', 'shps', 'sp_mpsp.geojson'))
    gdf = gdf.to_crs(epsg=4326)
    gdf = gdf[gdf['id_gaema'].notnull()]
    
    # Column with category
    col_categories = 'gaema_nome'
    
    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories])
    categories = list(categories)
    categories.sort()

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))    
    #color_polygon['1º Batalhão'] = '#ff4d4d'
    #color_polygon['2º Batalhão'] = '#4da6ff'
    #color_polygon['3º Batalhão'] = '#00b300'
    #color_polygon['4º Batalhão'] = '#ffcc99'
    #color_polygon1 = color_polygon
    
    # Popup
    gdf['popup_gaema'] = gdf.apply(popup_gaema, axis=1)
    
    # Geo
    lyr = folium.GeoJson(
        gdf,
        name='GAEMA',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': color_polygon[x['properties'][col_categories]],
            'color':     color_polygon[x['properties'][col_categories]],
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'gaema_nome'],
            aliases=['Munícipio', 'GAEMA'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        popup=folium.GeoJsonPopup(
            ['popup_gaema'],
            parse_html=False,
            max_width='400',
            show=False,
            labels=False,
            sticky=True,            
        ),
        zoom_on_click=False,
        embed=False,
        show=False,        
    )
    return lyr

# Add Field
def popup_gaema(row):
    #tel = str(row['telefone']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    #fax = str(row['fax']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    
    html = """
    <div>
    <p><b>{}</b> pertence ao:
    <h5><b>{}</b></h5></p>
    </div>
    """.format(
        '' if pd.isnull(row['municipio_nome']) else '{}'.format(row['municipio_nome']),
        '' if pd.isnull(row['gaema_nome']) else '{}'.format(row['gaema_nome']),
    )
    
    html = html.replace('\n','')
    html = re.sub('\s\s+' , ' ', html) # Remove Espaços no meio
    html = html.strip()
    return html

In [6]:
def add_lyr_redeprotetiva():
    # Input
    gdf = gpd.read_file(os.path.join('data', 'shps', 'sp_mpsp.geojson'))
    gdf = gdf.to_crs(epsg=4326)
    gdf = gdf[gdf['rp_nome'].notnull()]
    
    # Column with category
    col_categories = 'rp_nome'
    
    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories])
    categories = list(categories)
    categories.sort()

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))    
    #color_polygon['1º Batalhão'] = '#ff4d4d'
    #color_polygon['2º Batalhão'] = '#4da6ff'
    #color_polygon['3º Batalhão'] = '#00b300'
    #color_polygon['4º Batalhão'] = '#ffcc99'
    #color_polygon1 = color_polygon
    
    # Popup
    gdf['popup_redeprotetiva'] = gdf.apply(popup_redeprotetiva, axis=1)
    
    # Geo
    lyr = folium.GeoJson(
        gdf,
        name='Rede Protetiva',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': color_polygon[x['properties'][col_categories]],
            'color':     color_polygon[x['properties'][col_categories]],
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'rp_nome'],
            aliases=['Munícipio', 'Rede Protetiva'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        popup=folium.GeoJsonPopup(
            ['popup_redeprotetiva'],
            parse_html=False,
            max_width='400',
            show=False,
            labels=False,
            sticky=True,
        ),
        zoom_on_click=False,
        embed=False,
        show=False,
    )
    return lyr

# Add Field
def popup_redeprotetiva(row):
    #tel = str(row['telefone']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    #fax = str(row['fax']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    
    html = """
    <div>
    <p><b>{}</b> pertence ao:
    <h5><b>{}</b></h5></p>
    </div>
    """.format(
        '' if pd.isnull(row['municipio_nome']) else '{}'.format(row['municipio_nome']),
        '' if pd.isnull(row['rp_nome']) else '{}'.format(row['rp_nome']),
    )
    
    html = html.replace('\n', '')
    html = re.sub('\s\s+' , ' ', html) # Remove Espaços no meio
    html = html.strip()
    return html

<br>

# Map

In [7]:
def get_map(input_geojson):
    # Input
    gdf = gpd.read_file(input_geojson)
    gdf = gdf.to_crs(epsg=4326)
    sw = gdf.bounds[['miny', 'minx']].min().values.tolist()
    ne = gdf.bounds[['maxy', 'maxx']].max().values.tolist()
    bounds = [sw, ne]
    
    # Zoom
    min_zoom = 6
    max_zoom = 11
    
    # Create Map
    m = folium.Map(
        #zoom_start=6,
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        max_bounds=True,
        #zoom_delta=0.1,
        min_lat=bounds[0][0]*(101/100),
        min_lon=bounds[0][1]*(101/100),
        max_lat=bounds[1][0]*(99/100),
        max_lon=bounds[1][1]*(99/100),
        tiles=None,
    )
    
    # Add Layers
    m.add_child(add_lyr_cartodbpositron(min_zoom, max_zoom))
    #m.add_child(add_lyr_cpamb_pelotao())
    #m.add_child(add_lyr_cpamb_companhia())
    m.add_child(add_lyr_gaema())
    m.add_child(add_lyr_redeprotetiva())
    
    # Plugins
    m.fit_bounds(bounds)
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    folium.LayerControl(
        position='topright',
        collapsed=False,
    ).add_to(m)
    return m

In [8]:
# Map without Bounds
m = get_map(os.path.join('data', 'shps', 'sp_mpsp.geojson'))

# Results
os.makedirs('maps', exist_ok=True)
m.save(os.path.join('maps', 'mpsp_map.html'))
#m.save(os.path.join('..', '..', '..', 'case_django', 'divadmin', 'templates', 'cpamb_map.html'))
m